# Calibración usando GREG y Raking / IPF

En encuestas y estudios de muestreo, calibrar los pesos significa ajustarlos para que las distribuciones de la muestra coincidan con información externa conocida (por ejemplo, totales poblacionales por sexo, edad, ingresos, etc.). El objetivo es reducir sesgos y mejorar la representatividad de las estimaciones, partiendo de unos pesos base (de diseño o igual a 1) y un conjunto de restricciones que queremos cumplir.

En esta libreta se comparan dos métodos habituales:

__GREG (Generalized Regression Estimator) con enlace logístico__

__Idea__:
* Se plantea el problema como un ajuste por regresión, buscando los pesos más cercanos posible a los pesos base, sujetos a que las medias/totales ponderados de ciertas variables (marginales) coincidan con los valores poblacionales conocidos.
* Enlace logístico: Permite imponer límites superiores e inferiores a los pesos (por ejemplo, entre 0.3 y 3 veces el peso base), lo que evita valores extremos y mejora la estabilidad.

__Ventajas__
* 	Puede manejar variables continuas y categóricas simultáneamente.
* 	Ajusta todas las restricciones a la vez (no por iteraciones de margen en margen).
*	Minimiza explícitamente la desviación respecto a los pesos originales (menor efecto sobre la varianza).
* 	Integra bien con la teoría de varianza en encuestas.

__Desventajas__
* Requiere resolver un sistema lineal (posible inestabilidad numérica si hay colinealidad).
* Si las restricciones son incompatibles, puede no converger o necesitar relajaciones.

---
__Raking / IPF (Iterative Proportional Fitting)__

__Idea__:
Ajusta los pesos de forma multiplicativa, iterando sobre cada conjunto de restricciones (marginales) hasta que todas se cumplan dentro de una tolerancia.

__Funcionamiento básico__
*	Se calcula el total ponderado actual para cada categoría.
*	Se obtiene un factor de ajuste = (total objetivo / total actual).
*	Se multiplican los pesos de las observaciones en esa categoría por el factor.
*	Se repite para todos los márgenes, en ciclos, hasta converger.

__Ventajas__
*	Implementación sencilla y muy estable para márgenes puramente categóricos.
*	Muy rápido cuando el número de restricciones es bajo y los datos cubren bien todas las categorías.

__Desventajas__
*	Solo maneja variables categóricas (para continuas hay que discretizar).
*	Puede generar pesos extremos si hay celdas con pocos casos.
*	No minimiza de forma explícita la desviación respecto a los pesos originales.
*	Con muchas restricciones, la convergencia puede ser lenta.

---

__Calibración mediante Algoritmo Genético (Genetic Algorithm, GA)__

__Idea__;
Utiliza un enfoque evolutivo inspirado en la selección natural para encontrar el conjunto de pesos que mejor cumplen un conjunto de restricciones (marginales y/o conjuntas), manteniendo los pesos dentro de unos límites y lo más cercanos posible a los pesos base. En lugar de resolver el problema de forma analítica o iterativa por márgenes, el GA explora el espacio de soluciones mediante una población de candidatos y operadores de selección, cruce y mutación.

__Funcionamiento básico__
* 1.	Inicialización
	* 	Se genera una población inicial de soluciones (vectores de pesos) codificados como parámetros reales en un espacio transformado que garantiza que los pesos estén dentro de los límites [L, U].
	* 	Cada individuo parte de los pesos base con una ligera perturbación aleatoria.
* 2.	Evaluación (Fitness)
	*	Para cada individuo se calculan los pesos reales y se evalúa un función de aptitud (fitness) que combina:
	*	El error relativo frente a cada restricción objetivo (marginal y conjunta).
	*	Penalizaciones por desviarse en exceso de los pesos base (regularización).
	*	Un fitness menor indica una mejor solución.
* 3.	Selección
	*	Se eligen los individuos que pasarán a la siguiente generación mediante un mecanismo como el torneo (los mejores de grupos aleatorios pasan a reproducirse).
* 4.	Cruce (Crossover)
	*	Se combinan pares de individuos para generar descendencia, mezclando sus parámetros (por ejemplo, cruce por media ponderada).
* 5.	Mutación
	*	Se aplican pequeñas perturbaciones aleatorias a algunos parámetros para explorar nuevas regiones del espacio de soluciones.
* 6.	Elitismo
	*	Un número fijo de los mejores individuos se copia directamente a la siguiente generación para no perder soluciones ya encontradas.
* 7.	Iteración y parada
	*	El proceso de evaluación, selección, cruce y mutación se repite durante un número de generaciones o hasta que no haya mejora significativa en el fitness durante un número determinado de iteraciones.


__Ventajas__
* Puede manejar simultáneamente restricciones complejas: marginales, conjuntas, continuas, categóricas, o incluso con pesos diferentes en su importancia.
* Flexibilidad para incorporar objetivos blandos (no forzados al 100%) y penalizaciones personalizadas.
* No requiere que las restricciones sean perfectamente consistentes ni que haya soporte muestral completo para todas las categorías.
* Robusto frente a problemas no lineales o mal condicionados donde métodos como GREG pueden fallar.

__Desventajas__
* Coste computacional alto: requiere muchas evaluaciones de la función objetivo; no es ideal para muestras muy grandes.
* Resultados no deterministas (dependen de la semilla y parámetros del GA).
* Ajustar los hiperparámetros (tamaño de población, tasas de cruce/mutación, pesos de penalización) requiere experimentación.
* Puede converger a soluciones subóptimas si la exploración no es suficiente.

---

__Objeto del notebook: greg vs raking vs GA__

En este cuaderno:
1. Generamos un conjunto de datos sintético con variables categóricas: sexo, rango de edad, y banda de ingresos, más algunas distribuciones conjuntas conocidas.
2. Aplicamos GREG con enlace logístico para calibrar los pesos a los totales marginales especificados.
3.	Aplicamos Raking / IPF, tanto con solo márgenes como incluyendo también restricciones de distribuciones conjuntas (Age×Sex, Age×Income, Sex×Income).
4. Aplicamos un algoritmo genético con las características anteriores
5.	Medimos el ajuste:
	*	Errores medios y máximos absolutos frente a los objetivos.
	*	Divergencia KL.
	*	Comparación de cómo se ajustan las distribuciones conjuntas no restringidas.
6.	Comparamos el impacto en la distribución de pesos y en la precisión de las estimaciones.

La comparación permite ver cómo:
*	GREG tiende a producir pesos más “suaves” (menor dispersión) cuando hay muchas restricciones.
*	Raking puede clavar los márgenes exactos más fácilmente, pero a costa de mayor variabilidad en los pesos.
*	La inclusión de restricciones conjuntas convierte ambos métodos en ajustes más exigentes, pero con GREG es posible tratar esas restricciones como “blandas” si no se desean forzar.


In [200]:
import numpy as np
import pandas as pd
from IPython.display import display


# Creamos una serie de datos dummy

Incluimos también las probabilidades de las distribuciones marginales y conjuntas (en este caso usamos probs, pero podríamos estar usando otra magnitud como las sumas, etc).

Generamos 2000 instancias de forma aleatoria.

In [ ]:
# Specs
sex_levels = ["Male", "Female"]
income_bands = ["Low", "Lower-Mid", "Upper-Mid", "High"]
age_ranges = ["18-29", "30-44", "45-59", "60+"]

n_instances = 2000

# Define marginal probabilities
p_sex = np.array([0.48, 0.52])  # Male, Female
p_income = np.array([0.25, 0.25, 0.30, 0.20])
p_age = np.array([0.20, 0.30, 0.25, 0.25])

# Create a joint distribution for each pair (arbitrary but consistent with marginals)
# For Age x Sex (4x2)
joint_age_sex = np.array([
    [0.10, 0.10],  # 18-29: M, F
    [0.15, 0.15],  # 30-44
    [0.12, 0.13],  # 45-59
    [0.11, 0.14],  # 60+
])

# For Age x Income (4x4)
joint_age_income = np.array([
    [0.06, 0.05, 0.06, 0.03],  # 18-29
    [0.05, 0.07, 0.10, 0.08],  # 30-44
    [0.06, 0.06, 0.07, 0.06],  # 45-59
    [0.08, 0.07, 0.07, 0.03],  # 60+
])

# For Sex x Income (2x4)
joint_sex_income = np.array([
    [0.10, 0.12, 0.14, 0.12],  # Male
    [0.08, 0.10, 0.16, 0.18],  # Female
])

# Generate synthetic data that roughly follows marginals and joints
# We'll just sample independently for simplicity, then lightly adjust

sex = np.random.choice(sex_levels, size=n_instances, p=p_sex)
income = np.random.choice(income_bands, size=n_instances, p=p_income)
age = np.random.choice(age_ranges, size=n_instances, p=p_age)

df = pd.DataFrame({"Sex": sex, "IncomeBand": income, "AgeRange": age})

# Show the generated sample


In [202]:
display(df)

,Sex,IncomeBand,AgeRange
0,Female,Low,30-44
1,Male,High,45-59
2,Female,Lower-Mid,30-44
3,Female,High,30-44
4,Female,Upper-Mid,45-59
...,...,...,...
1995,Female,Low,18-29
1996,Male,Lower-Mid,60+
1997,Female,High,60+
1998,Female,Low,45-59


# Funciones de calibración

Usaremos dos funciones Greg (con alguna modificación para darle estabilidad al proceso de optimización a través de algebra lineal)

## Raking

In [203]:
# Calibración usando Raking / IPF (Iterative Proportional Fitting)

def rake_ipf(X_list, targets_list, d=None, 
             max_iter=100, tol=1e-8, trim=None, verbose=False):
    n = X_list[0].shape[0]
    if d is None:
        w = np.ones(n, dtype=float)
    else:
        w = np.array(d, dtype=float).copy()

    for it in range(max_iter):
        max_rel_err = 0.0

        for X, t in zip(X_list, targets_list):
            current = X.T @ w
            with np.errstate(divide='ignore', invalid='ignore'):
                f = np.where(current > 0, t / current, 1.0)
            # Multiply each row's weight by the factor for its category
            w *= (X * f).sum(axis=1)
            rel_err = np.max(np.abs(current - t) / (t + 1e-12))
            max_rel_err = max(max_rel_err, rel_err)

        if trim is not None:
            w = np.clip(w, trim[0], trim[1])

        if verbose:
            print(f"Iter {it+1}: max_rel_err = {max_rel_err:.3e}")
        if max_rel_err < tol:
            return w, {"iterations": it+1, "success": True, "max_rel_err": max_rel_err}

    return w, {"iterations": max_iter, "success": False, "max_rel_err": max_rel_err}

## GREG

In [204]:
def calibrate_logit_greg(X, X_pop, d, L=0.3, U=3.0, 
                         max_iter=100, tol=1e-8, ridge=1e-8, verbose=False):
    """
    Logistic-link calibration (logit GREG).
    
    Parameters
    ----------
    X : (n, p) array
        Auxiliaries for sample units (include dummies for post-strata, and/or continuous features).
    X_pop : (p,) array
        Known population totals for the same auxiliaries.
        If you want to match population means m_pop, pass X_pop = T * m_pop,
        where T is the population size (or desired total).
    d : (n,) array
        Base weights (e.g., design/nonresponse).
    L, U : float or (n,) arrays
        Lower/upper weight bounds as multipliers of d (0 < L < 1 < U).
    max_iter : int
        Newton iterations.
    tol : float
        Convergence tolerance on infinity-norm of moment mismatch.
    ridge : float
        Small Tikhonov regularization added to J for numerical stability.
    verbose : bool
        Print iteration diagnostics.
        
    Returns
    -------
    w : (n,) array
        Calibrated weights within [L*d, U*d].
    info : dict
        Diagnostics: lambda, iterations, success, final_mismatch.
    """
    X = np.asarray(X, dtype=float)
    X_pop = np.asarray(X_pop, dtype=float).ravel()
    d = np.asarray(d, dtype=float).ravel()
    n, p = X.shape
    assert X_pop.shape == (p,), "X_pop must be length p"
    assert d.shape == (n,), "d must be length n"
    L = np.broadcast_to(np.asarray(L, dtype=float), (n,))
    U = np.broadcast_to(np.asarray(U, dtype=float), (n,))
    assert np.all(L > 0) and np.all(L < 1) and np.all(U > 1), "Require 0<L<1<U"

    # Offset so that w=d when lambda=0
    # a = log((1-L)/(U-1)) works elementwise if L,U vary by unit
    a = np.log((1.0 - L) / (U - 1.0))

    lam = np.zeros(p)
    for it in range(max_iter):
        # alpha and weights
        eta = a + X @ lam
        # safeguard to avoid exact 0/1 in sigmoid
        eta = np.clip(eta, -30, 30)
        alpha = 1.0 / (1.0 + np.exp(-eta))
        w = d * (L + (U - L) * alpha)

        # moment mismatch
        g = X.T @ w - X_pop
        max_abs = np.max(np.abs(g))
        if verbose:
            print(f"Iter {it:02d}: mismatch_inf = {max_abs:.3e}")
        if max_abs < tol:
            return w, {"lambda": lam, "iterations": it, "success": True, "final_mismatch": g}

        # Jacobian: X^T diag( d*(U-L)*alpha*(1-alpha) ) X
        a1a = alpha * (1.0 - alpha)
        diag_w = d * (U - L) * a1a
        # build J via weighted Gram
        # J = X.T @ (diag(diag_w) @ X) but without forming the diag:
        J = X.T @ (diag_w[:, None] * X)
        # ridge for stability
        J[np.diag_indices_from(J)] += ridge

        # Newton step: solve J * step = g
        step = np.linalg.solve(J, g)
        lam = lam - step

    # If we reach here, no convergence
    return w, {"lambda": lam, "iterations": max_iter, "success": False, "final_mismatch": g}

Esta es una versión "segura" pero demasiado compleja

In [205]:
#
# Calibración usando GREG (Generalized Regression Estimation)
#

def _safe_sigmoid(z):
    # Stable sigmoid; handles large |z|
    z = np.clip(z, -40.0, 40.0)
    return 1.0 / (1.0 + np.exp(-z))

def calibrate_logit_greg_stable(
    X, X_pop, d, L=0.3, U=3.0,
    max_iter=100, tol=1e-8, ridge=1e-6,
    max_backtracks=8, step_shrink=0.5, verbose=False, standardize=True
):
    """
    Logistic-link calibration (logit GREG) with stability fixes:
    - column standardization
    - safe sigmoid & eta clipping
    - ridge-regularized Newton
    - backtracking line search
    Returns (w, info)
    """
    X = np.asarray(X, dtype=float)
    X_pop = np.asarray(X_pop, dtype=float).ravel()
    d = np.asarray(d, dtype=float).ravel()
    n, p = X.shape
    assert X_pop.shape == (p,)
    assert d.shape == (n,)

    # Broadcast bounds
    L = np.broadcast_to(np.asarray(L, dtype=float), (n,))
    U = np.broadcast_to(np.asarray(U, dtype=float), (n,))
    if not (np.all(L > 0) and np.all(L < 1) and np.all(U > 1)):
        raise ValueError("Require 0 < L < 1 < U (elementwise).")

    # Optional column scaling to reduce conditioning
    if standardize:
        col_scale = np.maximum(np.linalg.norm(X, axis=0), 1.0)  # L2 norm (avoid zeros)
        Xs = X / col_scale
        X_pop_s = X_pop / col_scale
    else:
        col_scale = np.ones(p)
        Xs = X
        X_pop_s = X_pop

    # Offset so w=d at lambda=0
    a = np.log((1.0 - L) / (U - 1.0))
    lam = np.zeros(p)

    # Helper: evaluate state
    def eval_state(lam_vec):
        # eta = a + Xs @ lam (lam is in scaled parameterization)
        eta = a + Xs @ lam_vec
        eta = np.clip(eta, -40.0, 40.0)
        alpha = _safe_sigmoid(eta)
        w = d * (L + (U - L) * alpha)
        # gradient (moment mismatch) in scaled coordinates
        g = Xs.T @ w - X_pop_s
        # diag for Jacobian
        a1a = np.clip(alpha * (1.0 - alpha), 1e-12, 0.25)
        diag_w = d * (U - L) * a1a
        return w, g, diag_w

    # Initial state
    w, g, diag_w = eval_state(lam)
    mismatch = np.max(np.abs(g))

    for it in range(max_iter):
        if verbose:
            print(f"[{it:02d}] mismatch_inf={mismatch:.3e}, ridge={ridge:.1e}")

        if not np.isfinite(mismatch) or np.isnan(mismatch):
            # escalate ridge and reset lambda if things went bad
            ridge *= 10.0
            lam[:] = 0.0
            w, g, diag_w = eval_state(lam)
            mismatch = np.max(np.abs(g))

        if mismatch < tol:
            # Done
            return w, {"lambda": lam / col_scale,  # unscale coeffs for reporting
                       "iterations": it,
                       "success": True,
                       "final_mismatch": g * col_scale,  # unscale gradient
                       "ridge": ridge}

        # Build Jacobian J = Xs.T @ diag(diag_w) @ Xs  (SPD-ish)
        J = Xs.T @ (diag_w[:, None] * Xs)

        # Ridge regularization (scaled by trace/p)
        tr = np.trace(J) / max(p, 1)
        J.flat[::p+1] += ridge + 1e-16 + 1e-12 * tr

        # Solve for Newton step: J * step = g
        try:
            step = np.linalg.solve(J, g)
        except np.linalg.LinAlgError:
            # add more ridge and retry
            ridge *= 10.0
            if verbose:
                print("  LinAlgError -> increasing ridge to", ridge)
            continue

        # Backtracking line search to reduce mismatch
        lam_new = lam - step
        w_new, g_new, diag_w_new = eval_state(lam_new)
        mismatch_new = np.max(np.abs(g_new))

        bt = 0
        while (not np.isfinite(mismatch_new)) or (mismatch_new >= mismatch) and bt < max_backtracks:
            lam_new = lam - (step_shrink ** (bt + 1)) * step
            w_new, g_new, diag_w_new = eval_state(lam_new)
            mismatch_new = np.max(np.abs(g_new))
            bt += 1

        if mismatch_new >= mismatch or not np.isfinite(mismatch_new):
            # Could not improve; increase ridge and try again
            ridge *= 10.0
            if verbose:
                print("  No improvement after backtracking; ridge ->", ridge)
            continue

        # Accept step
        lam = lam_new
        w, g, diag_w = w_new, g_new, diag_w_new
        mismatch = mismatch_new

    # Max iters reached
    return w, {"lambda": lam / col_scale,
               "iterations": max_iter,
               "success": False,
               "final_mismatch": g * col_scale,
               "ridge": ridge}

## Algoritmo evolutivo (genético)

In [206]:
# ---------- Helpers ----------
def _safe_sigmoid(z):
    z = np.clip(z, -40.0, 40.0)
    return 1.0 / (1.0 + np.exp(-z))

def _batch_weights(Z, d, L, U):
    # Z: (P, n) real-coded genome; returns weights (P, n)
    return d * (L + (U - L) * _safe_sigmoid(Z))

def _constraint_penalty(W, X_list, targets_list, eps=1e-12, weights=None):
    """
    Sum of weighted relative-squared errors across all constraints.
    W: (P, n) candidate weights
    X_list: list of (n, k_m) dummy matrices
    targets_list: list of (k_m,) arrays of totals
    weights: list of floats (per-constraint) to reweight penalties
    """
    P, n = W.shape
    total_pen = np.zeros(P)
    if weights is None:
        weights = [1.0] * len(X_list)

    for X, t, cw in zip(X_list, targets_list, weights):
        # Totals per candidate: (P, n) @ (n, k) -> (P, k)
        cur = W @ X
        # relative squared error vs targets, robust when t ~ 0
        denom = np.maximum(np.abs(t), eps)
        rse = ((cur - t) / denom) ** 2         # (P, k)
        total_pen += cw * np.mean(rse, axis=1) # average over categories
    return total_pen

def _kl_reg(W, d, eps=1e-12):
    """
    KL divergence per candidate between W and d (scaled): sum w*log(w/d) - (w-d)
    Return average per unit to normalize by n.
    """
    ratio = np.maximum(W / (d + eps), eps)
    return np.mean(W * (np.log(ratio)) - (W - d), axis=1)

# ---------- Genetic Algorithm Calibration ----------
def calibrate_ga(
    X_list, targets_list, *,
    d=None,                      # base weights (n,), default ones
    L=0.3, U=3.0,                # weight bounds as multipliers of d
    constraint_weights=None,     # list of floats, same length as X_list
    alpha=0.0,                   # L2 regularization toward base (on logit scale)
    beta=1e-4,                   # KL regularization toward base weights
    pop_size=200, generations=500,
    elite_frac=0.05, tournament_k=3,
    pc=0.9, pm=0.2, mut_sigma=0.15,  # crossover prob, mutation prob, mutation std for z
    patience=50, tol=1e-10,
    random_state=42, verbose=False
):
    """
    GA-based calibration of record-level weights.
    Constraints handled via penalties; weights stay in [L*d, U*d].

    Fitness = sum_m cw_m * mean( ((X_m^T w - t_m) / max(|t_m|, eps))^2 )
              + alpha * mean(z^2) + beta * KL(w || d)

    Returns
    -------
    w_best : (n,) array calibrated weights
    info   : dict with diagnostics
    """
    rng = np.random.default_rng(random_state)

    # Validate inputs
    n = X_list[0].shape[0]
    for X, t in zip(X_list, targets_list):
        assert X.shape[0] == n, "All X must have same number of rows (n)"
        assert X.shape[1] == t.shape[0], "Targets length must match X columns"

    if d is None:
        d = np.ones(n, dtype=float)
    else:
        d = np.asarray(d, float)

    # Broadcast bounds
    L = np.broadcast_to(np.asarray(L, float), (n,))
    U = np.broadcast_to(np.asarray(U, float), (n,))
    if not (np.all(L > 0) and np.all(L < 1) and np.all(U > 1)):
        raise ValueError("Require 0 < L < 1 < U (elementwise).")

    # Initialize population in z-space so that w starts near d
    # w = d*(L+(U-L)*sigmoid(z)); set z around logit((1-L)/(U-1)) + noise
    a = np.log((1.0 - L) / (U - 1.0))  # offset to make w=d for z=0 in mixed-bounds case
    # We’ll center at z=0 so w ~ midpoint; we’ll rely on GA to adjust
    Z = rng.normal(loc=0.0, scale=0.25, size=(pop_size, n))

    # Elitism count
    elite = max(1, int(np.round(elite_frac * pop_size)))

    # Vectorized evaluation
    def fitness(Zpop):
        W = _batch_weights(Zpop, d, L, U)
        pen = _constraint_penalty(W, X_list, targets_list, weights=constraint_weights)
        reg_l2 = alpha * np.mean(Zpop ** 2, axis=1) if alpha > 0 else 0.0
        reg_kl = beta * _kl_reg(W, d) if beta > 0 else 0.0
        return pen + reg_l2 + reg_kl, W

    # Selection: tournament
    def select_tournament(scores):
        idx = np.empty(pop_size, dtype=int)
        for i in range(pop_size):
            contenders = rng.integers(0, pop_size, size=tournament_k)
            best = contenders[np.argmin(scores[contenders])]
            idx[i] = best
        return idx

    # Crossover: blend (arithmetic)
    def crossover_blend(parents):
        P = parents.shape[0]
        idx = np.arange(P)
        rng.shuffle(idx)
        off = parents.copy()
        for i in range(0, P - 1, 2):
            if rng.random() < pc:
                a_idx, b_idx = idx[i], idx[i+1]
                alpha_c = rng.random()  # [0,1]
                child1 = alpha_c * parents[a_idx] + (1 - alpha_c) * parents[b_idx]
                child2 = alpha_c * parents[b_idx] + (1 - alpha_c) * parents[a_idx]
                off[a_idx], off[b_idx] = child1, child2
        return off

    # Mutation: Gaussian on z
    def mutate(Zpop):
        mask = rng.random(Zpop.shape) < pm
        Zpop = Zpop + mask * rng.normal(0.0, mut_sigma, size=Zpop.shape)
        # Clip z to keep sigmoid in a sensitive region
        return np.clip(Zpop, -8.0, 8.0)

    # Initial eval
    scores, W = fitness(Z)
    best_idx = int(np.argmin(scores))
    best_score = float(scores[best_idx])
    best_Z = Z[best_idx].copy()
    best_W = W[best_idx].copy()

    no_improve = 0
    history = [best_score]

    for g in range(generations):
        # Elitism
        elite_idx = np.argsort(scores)[:elite]
        elites_Z = Z[elite_idx].copy()

        # Selection
        sel_idx = select_tournament(scores)
        parents = Z[sel_idx]

        # Crossover + Mutation
        offspring = crossover_blend(parents)
        offspring = mutate(offspring)

        # Insert elites
        offspring[:elite] = elites_Z

        # Evaluate
        Z = offspring
        scores, W = fitness(Z)
        cur_best_idx = int(np.argmin(scores))
        cur_best = float(scores[cur_best_idx])

        if cur_best + tol < best_score:
            best_score = cur_best
            best_Z = Z[cur_best_idx].copy()
            best_W = W[cur_best_idx].copy()
            no_improve = 0
        else:
            no_improve += 1

        history.append(best_score)
        if verbose and (g % 10 == 0 or g == generations - 1):
            print(f"Gen {g:04d} | best fitness: {best_score:.6e}")

        if best_score < tol or no_improve >= patience:
            if verbose:
                print(f"Early stop at gen {g} (best={best_score:.3e}, no_improve={no_improve})")
            break

    info = {
        "success": best_score < tol or no_improve < patience,
        "best_fitness": best_score,
        "generations": g + 1,
        "history": np.array(history)
    }
    return best_W, info

## Función de evaluación de resultados

In [207]:
import numpy as np
import pandas as pd

# ---------- utilidades ----------
def normalize(p):
    p = np.asarray(p, dtype=float)
    s = p.sum()
    return p / s if s != 0 else p

def kl_div(p, q, eps=1e-12):
    p = normalize(p) + eps
    q = normalize(q) + eps
    return np.sum(p * np.log(p / q))

def compare_dist(target, before, after, name_order):
    """Devuelve tabla detallada (por categoría) y métricas agregadas antes/después."""
    def metrics(x, t):
        x = normalize(x); t = normalize(t)
        abs_diff = np.abs(x - t)
        rel_diff = abs_diff / (t + 1e-12)
        return pd.Series({
            "L1": abs_diff.sum(),
            "Linf": abs_diff.max(),
            "MAE": abs_diff.mean(),
            "RMSE": np.sqrt(np.mean((x - t) ** 2)),
            "RelMAE": rel_diff.mean(),
            "KL": kl_div(t, x)  # KL(target || x)
        })

    dfm = pd.DataFrame({
        "Category": name_order,
        "Target": normalize(target),
        "Before": normalize(before),
        "After": normalize(after),
        "AbsDiff_Before": np.abs(normalize(before) - normalize(target)),
        "AbsDiff_After": np.abs(normalize(after) - normalize(target)),
    })
    dfm_metrics = pd.DataFrame({
        "Before": metrics(before, target),
        "After": metrics(after, target)
    })
    return dfm, dfm_metrics

def weighted_counts(series, weights=None, levels=None):
    if levels is None:
        levels = series.unique().tolist()
    if weights is None:
        weights = np.ones(len(series))
    counts = pd.Series(0.0, index=levels)
    s = pd.Series(weights, index=series.index).groupby(series).sum()
    counts[s.index] = s.values
    return counts.loc[levels].values

def weighted_crosstab(a, b, weights=None, levels_a=None, levels_b=None):
    if levels_a is None: levels_a = a.unique().tolist()
    if levels_b is None: levels_b = b.unique().tolist()
    if weights is None: weights = np.ones(len(a))
    mat = np.zeros((len(levels_a), len(levels_b)), dtype=float)
    for i, la in enumerate(levels_a):
        mask_a = (a == la)
        for j, lb in enumerate(levels_b):
            mask = mask_a & (b == lb)
            mat[i, j] = weights[mask].sum()
    return mat, levels_a, levels_b

# ---------- función principal ----------
def report_calibration(
    df,
    weight_col: str,
    *,
    age_col="AgeRange",
    sex_col="Sex",
    inc_col="IncomeBand",
    # órdenes de niveles (para mantener consistencia con objetivos)
    age_levels=None,
    sex_levels=None,
    inc_levels=None,
    # objetivos marginales como proporciones (se escalan por T si no pasas T explícito)
    p_age=None,
    p_sex=None,
    p_income=None,
    # objetivos conjuntos como matrices de proporciones (se escalan por T)
    joint_age_sex=None,      # shape: (len(age_levels), len(sex_levels))
    joint_age_income=None,   # shape: (len(age_levels), len(inc_levels))
    joint_sex_income=None,   # shape: (len(sex_levels), len(inc_levels))
    # total de referencia; por defecto usa len(df)
    T=None,
    # impresión "bonita" en notebooks
    pretty_print=True
):
    """
    Genera resumen de ajuste marginal y conjunto antes vs. después,
    usando la columna de pesos indicada.
    Devuelve: summary_df (métricas), detailed_marginals (dict de tablas)
    """
    # Configurar niveles
    if age_levels is None:
        age_levels = df[age_col].unique().tolist()
    if sex_levels is None:
        sex_levels = df[sex_col].unique().tolist()
    if inc_levels is None:
        inc_levels = df[inc_col].unique().tolist()

    # Total de referencia
    if T is None:
        T = float(len(df))

    # Construir objetivos marginales (en totales)
    targets = {}
    if p_age is not None:
        targets["AgeRange"] = (np.asarray(p_age, float) * T, age_levels)
    if p_sex is not None:
        targets["Sex"] = (np.asarray(p_sex, float) * T, sex_levels)
    if p_income is not None:
        targets["IncomeBand"] = (np.asarray(p_income, float) * T, inc_levels)

    w = df[weight_col].to_numpy(float)

    # --- Marginales
    summary_rows = []
    detailed_tables = {}

    for var, (target_counts, levels) in targets.items():
        col = {"AgeRange": age_col, "Sex": sex_col, "IncomeBand": inc_col}[var]
        before_counts = weighted_counts(df[col], levels=levels)
        after_counts  = weighted_counts(df[col], weights=w, levels=levels)

        detail, metrics_tbl = compare_dist(target_counts, before_counts, after_counts, levels)
        detailed_tables[f"Marginal: {var}"] = detail

        summary_rows.append({
            "Distribution": f"{var} (marginal)",
            "Before_MAE":  metrics_tbl.loc["MAE", "Before"],
            "After_MAE":   metrics_tbl.loc["MAE", "After"],
            "Before_Linf": metrics_tbl.loc["Linf", "Before"],
            "After_Linf":  metrics_tbl.loc["Linf", "After"],
            "Before_KL":   metrics_tbl.loc["KL", "Before"],
            "After_KL":    metrics_tbl.loc["KL", "After"],
        })

    # --- Conjuntas (diagnóstico o hard constraints si las has usado)
    pair_targets = []
    if joint_age_sex is not None:
        pair_targets.append(((age_col, sex_col), (joint_age_sex, age_levels, sex_levels)))
    if joint_age_income is not None:
        pair_targets.append(((age_col, inc_col), (joint_age_income, age_levels, inc_levels)))
    if joint_sex_income is not None:
        pair_targets.append(((sex_col, inc_col), (joint_sex_income, sex_levels, inc_levels)))

    for (va, vb), (target_joint, la, lb) in pair_targets:
        M_before, _, _ = weighted_crosstab(df[va], df[vb], levels_a=la, levels_b=lb)
        M_after,  _, _ = weighted_crosstab(df[va], df[vb], weights=w, levels_a=la, levels_b=lb)

        # Compara en proporciones
        t = normalize(np.asarray(target_joint, float).ravel())
        b = normalize(M_before.ravel())
        a = normalize(M_after.ravel())

        summary_rows.append({
            "Distribution": f"{va} x {vb} (joint)",
            "Before_MAE":  float(np.mean(np.abs(b - t))),
            "After_MAE":   float(np.mean(np.abs(a - t))),
            "Before_Linf": float(np.max(np.abs(b - t))),
            "After_Linf":  float(np.max(np.abs(a - t))),
            "Before_KL":   float(kl_div(t, b)),
            "After_KL":    float(kl_div(t, a)),
        })

    summary_df = pd.DataFrame(summary_rows)

    if pretty_print:
        try:
            from IPython.display import display
            display(summary_df.round(6))
            for k, v in detailed_tables.items():
                display(k, v.round(6))
        except Exception:
            # entorno no interactivo
            print(summary_df.round(6).to_string(index=False))
            for k, v in detailed_tables.items():
                print("\n", k)
                print(v.round(6).to_string(index=False))

    return summary_df, detailed_tables

# Proceso de calibración

## Calibración con GREG

In [ ]:

# Marginal dummies
X_age = pd.get_dummies(df['AgeRange'], drop_first=False).values
X_sex = pd.get_dummies(df['Sex'], drop_first=False).values
X_inc = pd.get_dummies(df['IncomeBand'], drop_first=False).values

# Marginal targets (totals)
T = len(df)
# Base weights (all 1)
d = np.ones(T)
T_age = np.array(p_age) * T
T_sex = np.array(p_sex) * T
T_inc = np.array(p_income) * T

# Joint: Age × Sex
X_age_sex = pd.get_dummies(df[['AgeRange','Sex']].apply(tuple, axis=1), drop_first=False).values
# target joint totals in same col order as the dummy columns
joint_age_sex_t = joint_age_sex * T
T_age_sex = joint_age_sex_t.ravel()

# Joint: Age × Income
X_age_inc = pd.get_dummies(df[['AgeRange','IncomeBand']].apply(tuple, axis=1), drop_first=False).values
joint_age_income_t = joint_age_income * T
T_age_inc = joint_age_income_t.ravel()

# Joint: Sex × Income
X_sex_inc = pd.get_dummies(df[['Sex','IncomeBand']].apply(tuple, axis=1), drop_first=False).values
joint_sex_income_t = joint_sex_income * T
T_sex_inc = joint_sex_income_t.ravel()

# Combine all constraints
X_list = [X_age, X_sex, X_inc, X_age_sex, X_age_inc, X_sex_inc]
targets_list = [T_age, T_sex, T_inc, T_age_sex, T_age_inc, T_sex_inc]



Ejecutamos el proceso de calibración con Greg

In [ ]:
X = np.hstack(X_list)
X_pop = np.concatenate(targets_list)


calibrated_weights, calibration_attributes = calibrate_logit_greg(X, X_pop, d, L=0.05, U=10.0)

#calibrated_weights, calibration_attributes = calibrate_logit_greg_stable(X, X_pop, d, L=0.05, U=10.0)
df['Weight'] = calibrated_weights

/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/2155110067.py:51: RuntimeWarning: divide by zero encountered in matmul
  eta = a + X @ lam
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/2155110067.py:51: RuntimeWarning: overflow encountered in matmul
  eta = a + X @ lam
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/2155110067.py:51: RuntimeWarning: invalid value encountered in matmul
  eta = a + X @ lam
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/2155110067.py:58: RuntimeWarning: divide by zero encountered in matmul
  g = X.T @ w - X_pop
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/2155110067.py:58: RuntimeWarning: overflow encountered in matmul
  g = X.T @ w - X_pop
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/2155110067.py:58: RuntimeWarning: invalid value encountered in matmul
  g = X.T @ w - X_pop
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/2155110

## Ejecución con raking 2D / IPF

Realizamos la calibración usando proporciones, en este caso 

In [ ]:
# Run raking
w_rake, info_rake = rake_ipf(
    X_list=X_list,
    targets_list=targets_list,
    d=np.ones(len(df)),
    max_iter=10000,
    tol=1e-8,
    trim=None,
    verbose=False
)

df['Weight_Rake'] = w_rake

/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/3597979018.py:15: RuntimeWarning: divide by zero encountered in matmul
  current = X.T @ w
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/3597979018.py:15: RuntimeWarning: overflow encountered in matmul
  current = X.T @ w
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/3597979018.py:15: RuntimeWarning: invalid value encountered in matmul
  current = X.T @ w


## Calibración usando GA

In [ ]:
w_ga, info_ga = calibrate_ga(
    X_list, targets_list,
    d=d, L=0.3, U=3.0,
    constraint_weights=[1.0, 1.0, 1.0, 1.0],  # reweight constraints if needed
    alpha=1e-4,   # L2 on logit params (keeps z small)
    beta=1e-5,    # KL toward base weights (keeps w close to d)
    pop_size=300, generations=1000,
    elite_frac=0.08, tournament_k=3,
    pc=0.9, pm=0.2, mut_sigma=0.1,
    patience=80, tol=1e-10,
    random_state=7, verbose=False
)

df['Weight_GA'] = w_ga

/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/3933443838.py:25: RuntimeWarning: divide by zero encountered in matmul
  cur = W @ X
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/3933443838.py:25: RuntimeWarning: overflow encountered in matmul
  cur = W @ X
/var/folders/c8/4vzgjk5s1rd15q7tw2p8d9040000gn/T/ipykernel_35249/3933443838.py:25: RuntimeWarning: invalid value encountered in matmul
  cur = W @ X


## Evaluación de resultados

## Mostramos los totales de los pesos

In [255]:
display(df)

,Sex,IncomeBand,AgeRange,Weight,Weight_Rake,Weight_GA
0,Female,Low,30-44,10.0,0.898033,0.923437
1,Male,High,45-59,10.0,0.778324,1.276578
2,Female,Lower-Mid,30-44,10.0,1.288983,1.274257
3,Female,High,30-44,10.0,0.596819,1.096595
4,Female,Upper-Mid,45-59,10.0,0.904263,0.707695
...,...,...,...,...,...,...
1995,Female,Low,18-29,10.0,1.050938,1.032165
1996,Male,Lower-Mid,60+,10.0,1.422516,1.297342
1997,Female,High,60+,10.0,0.831730,0.987253
1998,Female,Low,45-59,10.0,0.868685,0.927539


In [256]:
# Mostramos unos estadísticos básicos de los pesos
# Summary statistics for the "Weight" column
weight_summary = df["Weight"].describe()
print('-'*50)
print('Estadísticos de los pesos (Weight):')
print('-'*50)
print(weight_summary)
print()
# Summary statistics for the "Weight_Rake" column
weight_rake_summary = df["Weight_Rake"].describe()
print('-'*50)
print('Estadísticos de los pesos (Weight_Rake):')
print('-'*50)
print(weight_rake_summary)


--------------------------------------------------
Estadísticos de los pesos (Weight):
--------------------------------------------------
count    2.000000e+03
mean     1.000000e+01
std      3.553602e-15
min      1.000000e+01
25%      1.000000e+01
50%      1.000000e+01
75%      1.000000e+01
max      1.000000e+01
Name: Weight, dtype: float64

--------------------------------------------------
Estadísticos de los pesos (Weight_Rake):
--------------------------------------------------
count    2000.000000
mean        1.000000
std         0.337313
min         0.394829
25%         0.750171
50%         0.898033
75%         1.227832
max         1.892492
Name: Weight_Rake, dtype: float64


In [257]:
# Summary statistics for the "Weight_Rake" column
weight_rake_summary = df["Weight_GA"].describe()
print('-'*50)
print('Estadísticos de los pesos (Weight_GA):')
print('-'*50)
print(weight_rake_summary)

--------------------------------------------------
Estadísticos de los pesos (Weight_GA):
--------------------------------------------------
count    2000.000000
mean        1.023259
std         0.260402
min         0.528481
25%         0.814406
50%         0.996149
75%         1.195960
max         2.144205
Name: Weight_GA, dtype: float64


### Evaluación de Greg

In [258]:
# Ejecuta el informe con la columna de pesos que quieras (p.ej., "Weight" o "Weight_Rake_Joint")
summary_df, detailed_marginals = report_calibration(
    df,
    weight_col="Weight",
    age_col="AgeRange", sex_col="Sex", inc_col="IncomeBand",
    age_levels=age_ranges, sex_levels=sex_levels, inc_levels=income_bands,
    p_age=p_age, p_sex=p_sex, p_income=p_income,
    joint_age_sex=joint_age_sex,
    joint_age_income=joint_age_income,
    joint_sex_income=joint_sex_income,
    T=len(df),
    pretty_print=True
)

,Distribution,Before_MAE,After_MAE,Before_Linf,After_Linf,Before_KL,After_KL
0,AgeRange (marginal),0.008500,0.008500,0.0170,0.0170,0.000945,0.000945
1,Sex (marginal),0.008500,0.008500,0.0085,0.0085,0.000145,0.000145
2,IncomeBand (marginal),0.011250,0.011250,0.0225,0.0225,0.001665,0.001665
3,AgeRange x Sex (joint),0.005875,0.005875,0.0100,0.0100,0.001531,0.001531
4,AgeRange x IncomeBand (joint),0.010938,0.010938,0.0315,0.0315,0.026952,0.026952
5,Sex x IncomeBand (joint),0.028125,0.028125,0.0530,0.0530,0.033025,0.033025


'Marginal: AgeRange'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,18-29,0.20,0.183,0.183,0.017,0.017
1,30-44,0.30,0.307,0.307,0.007,0.007
2,45-59,0.25,0.255,0.255,0.005,0.005
3,60+,0.25,0.255,0.255,0.005,0.005


'Marginal: Sex'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,Male,0.48,0.4715,0.4715,0.0085,0.0085
1,Female,0.52,0.5285,0.5285,0.0085,0.0085


'Marginal: IncomeBand'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,Low,0.25,0.2530,0.2530,0.0030,0.0030
1,Lower-Mid,0.25,0.2275,0.2275,0.0225,0.0225
2,Upper-Mid,0.30,0.3045,0.3045,0.0045,0.0045
3,High,0.20,0.2150,0.2150,0.0150,0.0150


In [259]:
# Ejecuta el informe con la columna de pesos que quieras (p.ej., "Weight" o "Weight_Rake_Joint")
summary_df, detailed_marginals = report_calibration(
    df,
    weight_col="Weight_Rake",
    age_col="AgeRange", sex_col="Sex", inc_col="IncomeBand",
    age_levels=age_ranges, sex_levels=sex_levels, inc_levels=income_bands,
    p_age=p_age, p_sex=p_sex, p_income=p_income,
    joint_age_sex=joint_age_sex,
    joint_age_income=joint_age_income,
    joint_sex_income=joint_sex_income,
    T=len(df),
    pretty_print=True
)

,Distribution,Before_MAE,After_MAE,Before_Linf,After_Linf,Before_KL,After_KL
0,AgeRange (marginal),0.008500,0.012131,0.0170,0.018104,0.000945,0.001327
1,Sex (marginal),0.008500,0.040000,0.0085,0.040000,0.000145,0.003202
2,IncomeBand (marginal),0.011250,0.025000,0.0225,0.050000,0.001665,0.007450
3,AgeRange x Sex (joint),0.005875,0.012696,0.0100,0.037351,0.001531,0.009225
4,AgeRange x IncomeBand (joint),0.010938,0.017877,0.0315,0.043763,0.026952,0.058862
5,Sex x IncomeBand (joint),0.028125,0.040000,0.0530,0.080000,0.033025,0.064696


'Marginal: AgeRange'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,18-29,0.20,0.183,0.192313,0.017,0.007687
1,30-44,0.30,0.307,0.318104,0.007,0.018104
2,45-59,0.25,0.255,0.256159,0.005,0.006159
3,60+,0.25,0.255,0.233425,0.005,0.016575


'Marginal: Sex'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,Male,0.48,0.4715,0.52,0.0085,0.04
1,Female,0.52,0.5285,0.48,0.0085,0.04


'Marginal: IncomeBand'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,Low,0.25,0.2530,0.22,0.0030,0.03
1,Lower-Mid,0.25,0.2275,0.30,0.0225,0.05
2,Upper-Mid,0.30,0.3045,0.30,0.0045,0.00
3,High,0.20,0.2150,0.18,0.0150,0.02


In [260]:
# Ejecuta el informe con la columna de pesos que quieras (p.ej., "Weight" o "Weight_Rake")
summary_df, detailed_marginals = report_calibration(
    df,
    weight_col="Weight_GA",
    age_col="AgeRange", sex_col="Sex", inc_col="IncomeBand",
    age_levels=age_ranges, sex_levels=sex_levels, inc_levels=income_bands,
    p_age=p_age, p_sex=p_sex, p_income=p_income,
    joint_age_sex=joint_age_sex,
    joint_age_income=joint_age_income,
    joint_sex_income=joint_sex_income,
    T=len(df),
    pretty_print=True
)

,Distribution,Before_MAE,After_MAE,Before_Linf,After_Linf,Before_KL,After_KL
0,AgeRange (marginal),0.008500,0.005316,0.0170,0.010633,0.000945,0.000414
1,Sex (marginal),0.008500,0.017944,0.0085,0.017944,0.000145,0.000644
2,IncomeBand (marginal),0.011250,0.035181,0.0225,0.070363,0.001665,0.014457
3,AgeRange x Sex (joint),0.005875,0.005933,0.0100,0.011711,0.001531,0.001486
4,AgeRange x IncomeBand (joint),0.010938,0.014304,0.0315,0.032893,0.026952,0.038994
5,Sex x IncomeBand (joint),0.028125,0.032728,0.0530,0.053211,0.033025,0.040914


'Marginal: AgeRange'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,18-29,0.20,0.183,0.189367,0.017,0.010633
1,30-44,0.30,0.307,0.308240,0.007,0.008240
2,45-59,0.25,0.255,0.250159,0.005,0.000159
3,60+,0.25,0.255,0.252234,0.005,0.002234


'Marginal: Sex'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,Male,0.48,0.4715,0.497944,0.0085,0.017944
1,Female,0.52,0.5285,0.502056,0.0085,0.017944


'Marginal: IncomeBand'

,Category,Target,Before,After,AbsDiff_Before,AbsDiff_After
0,Low,0.25,0.2530,0.256684,0.0030,0.006684
1,Lower-Mid,0.25,0.2275,0.274229,0.0225,0.024229
2,Upper-Mid,0.30,0.3045,0.229637,0.0045,0.070363
3,High,0.20,0.2150,0.239450,0.0150,0.039450
